# Low-Key Uber

#### A simple Q-Learning based solution to pick up a customer and drop her at destination 

#### The agent is rearded for successful drop-off of customer
#### The agent is penalised for wrong drop-off
#### A small penalty for every step the agent takes. This will allow the agent to learn the most optimal path

In [75]:
import gym
import numpy as np
from IPython.display import clear_output

#### The environment is a 5x5 grid. This gives us 25 different taxi locations. R, G, Y, B are the 4 locations on the 5x5 grid where a customer should be picked up and dropped.

In [85]:
# The Environment
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



#### The cab picks the customer from the stop which is blue in colour and drops off at the pink spot

In [77]:
env.action_space,env.observation_space

(Discrete(6), Discrete(500))

#### When we also account for one (1) additional passenger state of being inside the taxi, we can take all combinations of passenger locations and destination locations to come to a total number of states for our taxi environment; there's four (4) destinations and five (4 + 1) passenger locations.
#### So, our taxi environment has 5×5×5×4=500 total possible states.

#### There are 6 actions the cab can take: South(0), Norht(1), East(2), West(3), Pick-Up(4), Drop-Off(5)

In [87]:
env.s, env.P[env.s] # {action: [(probability, nextstate, reward, done)]}.

(384,
 {0: [(1.0, 484, -1, False)],
  1: [(1.0, 284, -1, False)],
  2: [(1.0, 384, -1, False)],
  3: [(1.0, 364, -1, False)],
  4: [(1.0, 384, -10, False)],
  5: [(1.0, 384, -10, False)]})

#### The possible action-reward policy for each state is give above.
#### For state=384, action=0(south), policy: 1.0(probability of action), 484(next state), -1(reward), False(check if done)

In [105]:
qTable = np.zeros((env.observation_space.n,env.action_space.n))

#### Q-Table of size 500x6 is built to store the Q-value for each state-action

### Q Learning Formula: 
#### Q(state, action)←(1−α)Q(state, action)+α(reward+γ*maxQ(next state, all actions)) 

In [106]:
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

#### Initialization of parameters
#### Alpha: Learning Rate, extent to which our Q-values are being updated
#### Gamma: Discount Factor, how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas, a discount factor of 0 makes our agent consider only immediate reward
#### Epsilon: help agent explore environmetn

In [107]:
"""Implement Q-Learning"""

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(qTable[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = qTable[state, action]
        next_max = np.max(qTable[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        qTable[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")


Episode: 100000
Training finished.



#### Implementation of Q-Learning.
#### Trained over 100,000 episodes

In [109]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 2
for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(qTable[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1
        clear_output(wait=True)
        env.render()
        time.sleep(0.5)
    total_penalties += penalties
    total_epochs += epochs

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
